In [ ]:
{-# LANGUAGE ScopedTypeVariables, ExistentialQuantification #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE RankNTypes, BangPatterns #-}
{-# LANGUAGE ForeignFunctionInterface #-}
import Control.Applicative
import Control.Monad
import Control.Monad.Primitive

import Data.List
import qualified Data.Vector.Generic as G
import           Data.Vector.Generic ((!))
import qualified Data.Vector.Unboxed as U

import Numeric.SpecFunctions
import Numeric.SpecFunctions.Extra (logGammaCorrection)
import Numeric.MathFunctions.Constants 
import Numeric.MathFunctions.Comparison
import Numeric.Polynomial
import Numeric.Series

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy

import Debug.Trace
import Text.Printf

:l NB/Plot

In [ ]:
-- | Compute the normalized lower incomplete gamma function
-- γ(/z/,/x/). Normalization means that γ(/z/,∞)=1
--
-- \[
-- \gamma(z,x) = \frac{1}{\Gamma(z)}\int_0^{x}t^{z-1}e^{-t}\,dt
-- \]
--
-- Uses Algorithm AS 239 by Shea.
incompleteGammaOld :: Double       -- ^ /z/ ∈ (0,∞)
                -> Double       -- ^ /x/ ∈ (0,∞)
                -> Double
incompleteGammaOld p x
    | isNaN p || isNaN x = m_NaN
    | x < 0 || p <= 0    = m_pos_inf
    | x == 0             = 0
    -- For very large `p' normal approximation gives <1e-10 error
    | p >= 2e5           = norm (3 * sqrt p * ((x/p) ** (1/3) + 1/(9*p) - 1))
    | p >= 500           = approx
    -- Dubious approximation
    | x >= 1e8           = 1
    | x <= 1 || x < p    = let a = p * log x - x - logGamma (p + 1)
                               g = a + log (pearson p 1 1)
                           in if g > limit then exp g else 0
    | otherwise          = let g = p * log x - x - logGamma p + log cf
                           in if g > limit then 1 - exp g else 1
  where
    -- CDF for standard normal distributions
    norm a = 0.5 * erfc (- a / m_sqrt_2)
    -- For large values of `p' we use 18-point Gauss-Legendre
    -- integration.
    approx
      | ans >  0           = 1 - ans
      | ans == 0 && x > p1 = 1 - ans
      | otherwise          = -ans
      where
        -- Set upper limit for integration
        xu | x > p1    =         (p1 + 11.5*sqrtP1) `max` (x + 6*sqrtP1)
           | otherwise = max 0 $ (p1 -  7.5*sqrtP1) `min` (x - 5*sqrtP1)
        s = U.sum $ U.zipWith go coefY coefW
        go y w = let t = x + (xu - x)*y
                 in w * exp( -(t-p1) + p1*(log t - lnP1) )
        ans = s * (xu - x) * exp( p1 * (lnP1 - 1) - logGamma p)
        --
        p1     = p - 1
        lnP1   = log  p1
        sqrtP1 = sqrt p1
    --
    pearson !a !c !g
        | c' <= tolerance = g'
        | otherwise       = pearson a' c' g'
        where a' = a + 1
              c' = c * x / a'
              g' = g + c'
    cf = let a = 1 - p
             b = a + x + 1
             p3 = x + 1
             p4 = x * b
         in contFrac a b 0 1 x p3 p4 (p3/p4)
    contFrac !a !b !c !p1 !p2 !p3 !p4 !g
        | abs (g - rn) <= min tolerance (tolerance * rn) = g
        | otherwise = contFrac a' b' c' (f p3) (f p4) (f p5) (f p6) rn
        where a' = a + 1
              b' = b + 2
              c' = c + 1
              an = a' * c'
              p5 = b' * p3 - an * p1
              p6 = b' * p4 - an * p2
              rn = p5 / p6
              f n | abs p5 > overflow = n / overflow
                  | otherwise         = n
    limit     = -88
    tolerance = 1e-14
    overflow  = 1e37


In [ ]:
-- Taken from BOOST
--
-- Calculates x^y - 1
powm1 :: Double -> Double -> Double
powm1 x y 
  | (abs x < 1 || abs y < 1) && abs p < 2 = expm1 p
  | otherwise                             = x**y - 1
  where
    p = log x * y

In [ ]:
-- Return approximation used
igammaImpl :: Double -> Double -> Double
igammaImpl z x
  | z <= 0 || x < 0 = error "Domain error"
  | x == 0          = -16
  | x < sqrt m_epsilon && z > 1 = -16
  | x < 0.5 = case () of
    _| (-0.4)/log x < z  -> -12
     | otherwise         -> -13
  | x < 1.1 = case () of
    _| 0.75*x < z -> -12
     | otherwise  -> -13
  | z > 20 && (  z > 200 && 20/z > mu*mu
              || abs mu < 0.4) = -15
  | x - (1 / (3 * x)) < z = -12
  | otherwise = -14
  where
    mu = (x - z) / z

# Roundtrip error

In [ ]:
igammaRoundtripError :: Double -> Double -> Double -> Double
igammaRoundtripError sc z p
  = traceShow (z,p) $ sc * m_epsilon/2 * (1 + abs (x * f' / p))
  where
    x  = invIncompleteGamma z p
    f' = exp ( log x * (z-1) - x - logGamma z)

plotErrorEstimate s z
  = toRenderable
  $ layout_title .~ ("Error estimate z=" ++ show z)
  $ plotFunctions
      [ \p -> logBase 10 $ relativeError p (incompleteGamma z $ (traceShow p $ invIncompleteGamma z p))
      , logBase 10 . igammaRoundtripError s z
      , logBase 10 . const (m_epsilon/2)
      ] (0,1)

In [ ]:
plotErrorEstimate 16 0.1
plotErrorEstimate 16 1
plotErrorEstimate 16 10
plotErrorEstimate 16 100
plotErrorEstimate 16 10001

In [ ]:
invIncompleteGamma 80 1
incompleteGamma 80 (1/0)

# Incomplete gamma precision

$$\gamma(1,x) = 1 - \exp(-x)$$

In [ ]:
toRenderable $
  plotFunctions
    [ \x -> logBase 10 $ relativeError (1 - exp (-x)) (incompleteGamma 1 x) 
    , logBase 10 . const (m_epsilon/2)
    ] (0,10)


In [ ]:
toRenderable $
  plotFunctions
    [ \x -> logBase 10 $ relativeError (erf (sqrt x)) (incompleteGamma 0.5 x) 
    , logBase 10 . const (m_epsilon/2)
    ] (0,20)

In [ ]:
igammaMPMathPlot2 z fname = do
  xs <- readFile fname
  let vals = map ((\[x,y] -> (read x :: Double, read y :: Double)) . words) $ lines xs
  --
  return $ toRenderable $    
    let isOK x = True --  x > 1e-1 && x < 1e6
        pts = [ (logBase 10 x, d)
              | (x,y) <- vals
              , let d0 = relativeError y (igamma z x)
                    d  = if d0 == 0 then -16.1 else logBase 10 d0
              , not (isNaN d)
              , isOK x
              ]
        ptsBase = [(logBase 10 x,logBase 10 (m_epsilon/2)) 
                  | (x,y) <- vals
                  , isOK x
                  ]
        ptsImpl = [(logBase 10 x, igammaImpl z x)
                  | (x,y) <- vals
                  , isOK x
                  ]
    in layout_plots .~ [ toPlot $ plot_lines_values .~ [pts] $ def
                       , toPlot $ plot_lines_values .~ [ptsBase]
                                $ plot_lines_style .~ (line_color .~ opaque red $ def)
                                $ def
                       , toPlot $ plot_lines_values .~ [ptsImpl]
                                $ plot_lines_style .~ (line_color .~ opaque green $ def)
                                $ def
                       ]
      $ layout_title .~ ("incomplete gamma: z="++show z)
      $ def
      


igammaMPMathPlot2 0.1  "../data/igamma-0.1.dat"
igammaMPMathPlot2 1    "../data/igamma-1.0.dat"
igammaMPMathPlot2 10   "../data/igamma-10.dat"
igammaMPMathPlot2 100  "../data/igamma-100.dat"
igammaMPMathPlot2 1000 "../data/igamma-1000.dat"
--igammaMPMathPlot2 10000 "../data/igamma-10000.dat"

In [ ]:
igammaMPMathPlot z fname = do
  xs <- readFile fname
  let vals = map ((\[x,y] -> (read x :: Double, read y :: Double)) . words) $ lines xs
  --
  return $ toRenderable $
    let pts = [ (logBase 10 x, d)
              | (x,y) <- vals
              , let d = logBase 10 $ relativeError y (incompleteGamma z x)
              , not (isNaN d || isInfinite d)
              ]
    in layout_plots .~ [ toPlot $ plot_lines_values .~ [pts] $ def
                       , toPlot $ plot_lines_values .~ [[(logBase 10 x,logBase 10 (m_epsilon/2)) | (x,_) <- vals]]
                                $ plot_lines_style .~ (line_color .~ opaque red $ def)
                                $ def
                       ]
      $ layout_title .~ ("incomplete gamma: z="++show z)
      $ def

In [ ]:
igammaMPMathPlot 0.1 "data/igamma-0.1.dat"
igammaMPMathPlot 1.0 "data/igamma-1.0.dat"
igammaMPMathPlot 10  "data/igamma-10.dat"

In [ ]:
incompleteGamma 10 1e-5

In [ ]:
igamma 10 0.2

In [ ]:
igammaMPMathPlotOrig z fname = do
  xs <- readFile fname
  let vals = map ((\[x,y] -> (read x :: Double, read y :: Double)) . words) $ lines xs
  --
  return $ toRenderable $    
    let isOK x = True --  x > 1e-1 && x < 1e6
        pts = [ (logBase 10 x, d)
              | (x,y) <- vals
              , let d0 = relativeError y (incompleteGamma z x)
                    d  = if d0 == 0 then -16.1 else logBase 10 d0
              , not (isNaN d)
              , isOK x
              ]
        ptsBase = [(logBase 10 x,logBase 10 (m_epsilon/2)) 
                  | (x,y) <- vals
                  , isOK x
                  ]
        ptsImpl = [(logBase 10 x, igammaImpl z x)
                  | (x,y) <- vals
                  , isOK x
                  ]
    in layout_plots .~ [ toPlot $ plot_lines_values .~ [pts] $ def
                       , toPlot $ plot_lines_values .~ [ptsBase]
                                $ plot_lines_style .~ (line_color .~ opaque red $ def)
                                $ def
                       , toPlot $ plot_lines_values .~ [ptsImpl]
                                $ plot_lines_style .~ (line_color .~ opaque green $ def)
                                $ def
                       ]
      $ layout_title .~ ("incomplete gamma: z="++show z)
      $ def
      


igammaMPMathPlotOrig 0.1  "../data/igamma-0.1.dat"
igammaMPMathPlotOrig 1    "../data/igamma-1.0.dat"
igammaMPMathPlotOrig 10   "../data/igamma-10.dat"
igammaMPMathPlotOrig 100  "../data/igamma-100.dat"
igammaMPMathPlotOrig 1000 "../data/igamma-1000.dat"
igammaMPMathPlotOrig 10000 "../data/igamma-10000.dat"

In [ ]:
igammaMPMathPlotCmp z fname = do
  xs <- readFile fname
  let vals = map ((\[x,y] -> (read x :: Double, read y :: Double)) . words) $ lines xs
  --
  return $ toRenderable $    
    let isOK x = True --  x > 1e-1 && x < 1e6
        pts = [ (logBase 10 x, d)
              | (x,y) <- vals
              , let d0 = relativeError y (igamma z x)
                    d  = if d0 == 0 then -16.1 else logBase 10 d0
              , not (isNaN d)
              , isOK x
              ]
        ptsOld = [ (logBase 10 x, d)
              | (x,y) <- vals
              , let d0 = relativeError y (incompleteGamma z x)
                    d  = if d0 == 0 then -16.1 else logBase 10 d0
              , not (isNaN d)
              , isOK x
              ]
        ptsBase = [(logBase 10 x,logBase 10 (m_epsilon/2)) 
                  | (x,y) <- vals
                  , isOK x
                  ]
    in layout_plots .~ [ toPlot $ plot_lines_values .~ [pts] $ def
                       , toPlot $ plot_lines_values .~ [ptsBase]
                                $ plot_lines_style .~ (line_color .~ opaque red $ def)
                                $ def
                       , toPlot $ plot_lines_values .~ [ptsOld]
                                $ plot_lines_style .~ (line_color .~ opaque green $ def)
                                $ def
                       ]
      $ layout_title .~ ("incomplete gamma: z="++show z)
      $ def
      


igammaMPMathPlotCmp 0.1  "../data/igamma-0.1.dat"
igammaMPMathPlotCmp 1    "../data/igamma-1.0.dat"
igammaMPMathPlotCmp 10   "../data/igamma-10.dat"
igammaMPMathPlotCmp 100  "../data/igamma-100.dat"
igammaMPMathPlotCmp 1000 "../data/igamma-1000.dat"
igammaMPMathPlotCmp 10000 "../data/igamma-10000.dat"

In [ ]:
sumSeries $ recip <$> enumSeriesFrom 1

In [ ]:
igamma 100 1e308

In [ ]:
logGamma 1e308